In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [18,10]

In [3]:
non_numeric = ['BMI_class', 'Height_class', 'Gender', 'Component', 'Branch']
regression = ['BMI_class', 'Height_class', 'Gender', 'Component', 'Branch', 'BMI']

def load_ansur(cols_to_drop, test_size, model_type='class'):
    df_m = pd.read_csv('data/ANSUR_II_MALE.csv')
    df_f = pd.read_csv('data/ANSUR_II_FEMALE.csv')
    ansur_df = pd.concat([df_m, df_f], axis=0)
    
    if model_type == 'class': 
        X = ansur_df.drop(non_numeric, axis=1)
        y = ansur_df['Gender']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    elif model_type == 'reg':
        X = ansur_df.drop(regression, axis=1)
        y = ansur_df["BMI"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
    else:
        print('please specify model type')
        
    return X, y, X_train, X_test, y_train, y_test

In [5]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, .3)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

### Creating a logistic regression model

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train_std, y_train)

X_test_std = scaler.transform(X_test)

In [7]:
y_pred = lr.predict(X_test_std)
print(accuracy_score(y_test, y_pred))

0.99945085118067


### Inspecting the feature coefficients 

In [8]:
print(lr.coef_[0][:10])

[ 0.20971913  0.09812458 -0.00409289 -0.33073854 -0.07887948  0.0678161
  0.28519533  0.73488134  0.68593631 -0.67481596]


In [9]:
coef_dict = dict(zip(X.columns, abs(lr.coef_[0])))

{k: v for i, (k, v) in enumerate(coef_dict.items()) if i < 10}

{'abdominalextensiondepthsitting': 0.20971912638589177,
 'acromialheight': 0.0981245756541675,
 'acromionradialelength': 0.004092886020990031,
 'anklecircumference': 0.3307385353616708,
 'axillaheight': 0.07887948108127388,
 'balloffootcircumference': 0.0678161023773195,
 'balloffootlength': 0.28519533222517146,
 'biacromialbreadth': 0.7348813437044783,
 'bicepscircumferenceflexed': 0.6859363095229345,
 'bicristalbreadth': 0.6748159648131341}

In [10]:
low_coef = {k: v for k, v in coef_dict.items() if v < .401}

cols = [k for k, v in low_coef.items()]

In [11]:
X.drop(cols, axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lr.fit(scaler.fit_transform(X_train), y_train)

print(accuracy_score(y_test, lr.predict(scaler.transform(X_test))))

1.0


In [13]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, .3)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [14]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=5, verbose=0)
rfe.fit(X_train_std, y_train)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)

In [15]:
X.columns[rfe.support_]

Index(['biacromialbreadth', 'bicepscircumferenceflexed',
       'buttockcircumference', 'chestheight', 'neckcircumference'],
      dtype='object')

In [16]:
#print(dict(zip(X.columns, rfe.ranking_)))

In [17]:
print(accuracy_score(y_test, rfe.predict(X_test_std)))

0.9928610653487095


In [18]:
def load_pima(cols_to_drop):
    df = pd.read_csv('data/PimaIndians.csv')

    X = df.drop(cols_to_drop, axis=1)
    y = df['test']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X, y, X_train, X_test, y_train, y_test

X, y, X_train, X_test, y_train, y_test = load_pima('test')

scaler = StandardScaler()
lr = LogisticRegression()

In [19]:
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

# Fit the logistic regression model on the scaled training data
lr.fit(X_train_std, y_train)

# Scale the test features
X_test_std = scaler.transform(X_test)

# Predict diabetes presence on the scaled test set
y_pred = lr.predict(X_test_std)

# Prints accuracy metrics and feature coefficients
print("{0:.1%} accuracy on test set.".format(accuracy_score(y_test, y_pred))) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

80.5% accuracy on test set.
{'pregnant': 0.37, 'glucose': 0.95, 'diastolic': 0.06, 'triceps': 0.12, 'insulin': 0.12, 'bmi': 0.39, 'family': 0.34, 'age': 0.37}


In [20]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Create the RFE with a LogisticRegression estimator and 3 features to select
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

# Fits the eliminator to the data
rfe.fit(X_train_std, y_train)

# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

# Print the features that are not eliminated
print(X.columns[rfe.support_])

# Calculates the test set accuracy
acc = accuracy_score(y_test, rfe.predict(X_test_std))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
{'pregnant': 6, 'glucose': 1, 'diastolic': 5, 'triceps': 3, 'insulin': 4, 'bmi': 1, 'family': 1, 'age': 2}
Index(['glucose', 'bmi', 'family'], dtype='object')
75.4% accuracy on test set.


In [22]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, .3)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [24]:
print(accuracy_score(y_test, rf.predict(X_test)))

0.9934102141680395


In [25]:
print(rf.feature_importances_)

[0.00125776 0.00053666 0.00037606 0.00119473 0.00113228 0.00885913
 0.01463319 0.07866825 0.00610072 0.00609354 0.01422648 0.01935199
 0.00069705 0.00152239 0.00098723 0.01470802 0.00386368 0.00060055
 0.0023732  0.00134111 0.00183741 0.01098181 0.00073335 0.00079454
 0.00564815 0.01353659 0.00054052 0.00651752 0.00073437 0.00064342
 0.00089249 0.00329732 0.00056142 0.00384012 0.00102085 0.00506609
 0.00439965 0.01356455 0.00691025 0.00169881 0.00063661 0.04414603
 0.08670947 0.00073278 0.00161244 0.00084052 0.00063595 0.03913689
 0.00063302 0.0131046  0.02688284 0.00105357 0.00060237 0.00363395
 0.02018483 0.00082139 0.00033041 0.00147863 0.00727611 0.0052866
 0.0020804  0.1213228  0.10808645 0.00103983 0.00041388 0.00729979
 0.00984216 0.08227404 0.00449031 0.00084629 0.00262345 0.03454954
 0.00060869 0.00495518 0.00106514 0.00058031 0.00778551 0.00105519
 0.00047585 0.00058901 0.00062611 0.00094411 0.0008168  0.02200306
 0.0012322  0.00163723 0.00131147 0.00058765 0.00050649 0.04838

In [26]:
mask = rf.feature_importances_ > 0.03
print(mask)

[False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True  True False False False False  True
 False False False False False False False False False False False False
 False  True  True False False False False  True False False False  True
 False False False False False False False False False False False False
 False False False False False  True False False False False]


In [27]:
X_reduced = X_train.loc[:, mask]
print(X_reduced.columns)

Index(['biacromialbreadth', 'handbreadth', 'handcircumference',
       'heelanklecircumference', 'neckcircumference', 'neckcircumferencebase',
       'shouldercircumference', 'sleevelengthspinewrist',
       'wristcircumference'],
      dtype='object')


### RFE with random forests

In [28]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=RandomForestClassifier(), 
          n_features_to_select=6, step=10, 
            verbose=1)

rfe.fit(X_train, y_train)

Fitting estimator with 94 features.
Fitting estimator with 84 features.
Fitting estimator with 74 features.
Fitting estimator with 64 features.
Fitting estimator with 54 features.
Fitting estimator with 44 features.
Fitting estimator with 34 features.
Fitting estimator with 24 features.
Fitting estimator with 14 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False),
    n_features_to_select=6, step=10, verbose=1)

In [29]:
print(X_train.columns[rfe.support_])

Index(['biacromialbreadth', 'handcircumference', 'neckcircumference',
       'neckcircumferencebase', 'shouldercircumference', 'wristcircumference'],
      dtype='object')


In [30]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

In [31]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

# Print the importances per feature
print(dict(zip(X_train.columns, rf.feature_importances_.round(2))))

# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc))

{'pregnant': 0.07, 'glucose': 0.24, 'diastolic': 0.07, 'triceps': 0.1, 'insulin': 0.15, 'bmi': 0.12, 'family': 0.12, 'age': 0.14}
79.7% accuracy on test set.


In [32]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Prints out the mask
print(mask)

[False  True False False False False False False]


In [33]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Apply the mask to the feature dataset X
reduced_X = X_train.loc[:, mask]

# prints out the selected column names
print(reduced_X.columns)

Index(['glucose'], dtype='object')


In [34]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

In [35]:
# Wrap the feature eliminator around the random forest model
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=2, step=2, verbose=1)

# Fit the model to the training data
rfe.fit(X_train, y_train)

# Create a mask using an attribute of rfe
mask = rfe.support_

# Apply the mask to the feature dataset X and print the result
reduced_X = X.loc[:, mask]
print(reduced_X.columns)

Fitting estimator with 8 features.
Fitting estimator with 6 features.
Fitting estimator with 4 features.
Index(['glucose', 'insulin'], dtype='object')


In [36]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, .3, 'reg')

In [37]:
from sklearn.linear_model import Lasso

In [38]:
X_train_std = scaler.fit_transform(X_train)

# Create the Lasso model
la = Lasso()

# Fit it to the standardized training data
la.fit(X_train_std, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [39]:
# Transform the test set with the pre-fitted scaler
X_test_std = scaler.transform(X_test)

# Calculate the coefficient of determination (R squared) on X_test_std
r_squared = la.score(X_test_std, y_test)
print("The model can predict {0:.1%} of the variance in the test set.".format(r_squared))

# Create a list that has True values when coefficients equal 0
zero_coef = la.coef_ == 0

# Calculate how many features have a zero coefficient
n_ignored = sum(zero_coef)
print("The model has ignored {} out of {} features.".format(n_ignored, len(la.coef_)))

The model can predict 82.9% of the variance in the test set.
The model has ignored 83 out of 93 features.


In [40]:
alphas = [1, 0.5, 0.1, 0.01]
for a in alphas:
    # Find the highest alpha value with R-squared above 98%
    la = Lasso(alpha=a, random_state=0)

    # Fits the model and calculates performance stats
    la.fit(X_train_std, y_train)
    r_squared = la.score(X_test_std, y_test)
    n_ignored_features = sum(la.coef_ == 0)

    # Print peformance stats 
    print("The model can predict {0:.1%} of the variance in the test set.".format(r_squared))
    print("{} out of {} features were ignored.".format(n_ignored_features, len(la.coef_)))

The model can predict 82.9% of the variance in the test set.
83 out of 93 features were ignored.
The model can predict 90.8% of the variance in the test set.
82 out of 93 features were ignored.
The model can predict 98.5% of the variance in the test set.
69 out of 93 features were ignored.
The model can predict 99.3% of the variance in the test set.
53 out of 93 features were ignored.


### Combining feature selectors 
#### Taking a step back
- Random forest is combination of decision trees 
- We can use combination of models for feature selection too

In [41]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric, 0.25, 'reg')

In [42]:
from sklearn.linear_model import LassoCV

lcv = LassoCV()
lcv.fit(X_train, y_train)

lcv.score(X_test, y_test)

0.9895676219048437

In [43]:
lcv_mask = lcv.coef_ != 0
sum(lcv_mask)

38

In [44]:
from sklearn.ensemble import RandomForestRegressor

rfe_rf = RFE(estimator=RandomForestRegressor(),
            n_features_to_select=38, step=5, verbose=5)

rfe_rf.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 88 features.
Fitting estimator with 83 features.
Fitting estimator with 78 features.
Fitting estimator with 73 features.
Fitting estimator with 68 features.
Fitting estimator with 63 features.
Fitting estimator with 58 features.
Fitting estimator with 53 features.
Fitting estimator with 48 features.
Fitting estimator with 43 features.


RFE(estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                    criterion='mse', max_depth=None,
                                    max_features='auto', max_leaf_nodes=None,
                                    max_samples=None, min_impurity_decrease=0.0,
                                    min_impurity_split=None, min_samples_leaf=1,
                                    min_samples_split=2,
                                    min_weight_fraction_leaf=0.0,
                                    n_estimators=100, n_jobs=None,
                                    oob_score=False, random_state=None,
                                    verbose=0, warm_start=False),
    n_features_to_select=38, step=5, verbose=5)

In [45]:
rf_mask = rfe_rf.support_

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

rfe_gb = RFE(estimator=GradientBoostingRegressor(), 
            n_features_to_select=38, step=5, verbose=1)

rfe_gb.fit(X_train, y_train)

Fitting estimator with 93 features.
Fitting estimator with 88 features.
Fitting estimator with 83 features.
Fitting estimator with 78 features.
Fitting estimator with 73 features.
Fitting estimator with 68 features.
Fitting estimator with 63 features.
Fitting estimator with 58 features.
Fitting estimator with 53 features.
Fitting estimator with 48 features.
Fitting estimator with 43 features.


In [ ]:
gb_mask = rfe_gb.support_

In [ ]:
votes =np.sum([lcv_mask, rf_mask, gb_mask], axis=0)
print(votes)

In [ ]:
mask = votes >= 2

In [ ]:
reduced_X = X.loc[:, mask]
#reduced_X

In [ ]:
!jupyter nbconvert --to html 3_Screening_for_model_accuracy.ipynb

In [ ]:
!../gitbsh > /dev/null 2>&1